<center><img src="dog.jpg" width=450></center>


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Photo credit: [SeventyFour](https://www.istockphoto.com/en/portfolio/SeventyFour?mediatype=photography)

### What is a database?

Databases store and organize data electronically. The data is housed in objects called tables, with data organized into rows and columns (or records and fields). 

A relational database defines relationships between tables of data inside a database.

Although they can look similar to a spreadsheet, databases can store more data, more securely, and allow multiple users to access the same database.

### Our data

This project will use data from the [Austin Animal Shelter](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238/data_preview).
In this case, the data is saved to this DataLab workbook as a CSV file, but you'll query it just like you would a database!

Here is copy of the data scription from the Animal Shelter website and what you'd expect to find in the data:

|Column Name|Description|Data Type|
|-----------|-----------|---------|
|`animal_id`|Identifier|Text|
|`name`|Animal name, if provided|Text|
|`datetime`|Date of outcome|Floating Timestamp|
|`monthyear`|Month and year of outcome|Floating Timestamp|
|`date_of_birth`|Animal's date of birth|Floating Timestamp|
|`outcome_type`|Details of the outcome|Text|
|`outcome_subtype`|Additional outcome details|Text|
|`animal_type`|Type of animal|Text|
|`sex_upon_outcome`|Details on whether the animal was neutered or intact|Text|
|`age_upon_outcome`|Age of animal at time of outcome|Text|
|`breed`|Animal breed|Text|
|`color`|Animal color|Text|

### Step 1: Explore the data

Before you dig into the data, it's always a good idea to explore it to see if it is as expected. Does the data match what was outlined above? Try querying the data to view a snapshot of the first 10 rows.

In [1]:
SELECT *
FROM 'animal_shelter.csv'
LIMIT 10;


,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color
0,A882831,*Hamilton,2023-07-01 18:12:00+00:00,Jul 2023,2023-03-25 00:00:00+00:00,Adoption,None,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Black/White
1,A794011,Chunk,2019-05-08 18:20:00+00:00,May 2019,2017-05-02 00:00:00+00:00,Rto-Adopt,None,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
2,A776359,Gizmo,2018-07-18 16:02:00+00:00,Jul 2018,2017-07-12 00:00:00+00:00,Adoption,None,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
3,A821648,None,2020-08-16 11:38:00+00:00,Aug 2020,2019-08-16 00:00:00+00:00,Euthanasia,None,Other,Unknown,1 year,Raccoon,Gray
4,A720371,Moose,2016-02-13 17:59:00+00:00,Feb 2016,2015-10-08 00:00:00+00:00,Adoption,None,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
5,A674754,None,2014-03-18 11:47:00+00:00,Mar 2014,2014-03-12 00:00:00+00:00,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
6,A659412,Princess,2020-10-05 14:37:00+00:00,Oct 2020,2013-03-24 00:00:00+00:00,Adoption,None,Dog,Spayed Female,7 years,Chihuahua Shorthair Mix,Brown
7,A814515,Quentin,2020-05-06 07:59:00+00:00,May 2020,2018-03-01 00:00:00+00:00,Adoption,Foster,Dog,Neutered Male,2 years,American Foxhound/Labrador Retriever,White/Brown
8,A902098,None,2024-07-18 12:07:00+00:00,Jul 2024,2024-04-10 00:00:00+00:00,Adoption,Foster,Cat,Neutered Male,3 months,Domestic Shorthair,Blue Tabby
9,A860161,*Lalo,2022-07-19 15:53:00+00:00,Jul 2022,2022-05-04 00:00:00+00:00,Adoption,None,Cat,Neutered Male,2 months,Domestic Shorthair,Brown Tabby


It's also helpful to understand how big the dataset is. How may rows (or records) are there in this data? Query the file to find out.

In [1]:
SELECT COUNT(animal_id)
FROM 'animal_shelter.csv';

,count(animal_id)
0,169720


Your turn! Can you think of another way to count the rows (or records)?

In [1]:
SELECT COUNT(*) AS total_rows
FROM 'animal_shelter.csv';

,total_rows
0,169720


### Step 2: Get deeper and answer questions about the data

Let's explore the data further. You're probably curious to know what animals came through this shelter so let's start there. How many different animal types has this shelter taken in?

In [2]:
SELECT COUNT(DISTINCT animal_type)
FROM 'animal_shelter.csv';

,count(DISTINCT animal_type)
0,5


Now you know how many different animal types there were, can you also find out some more information? What were those animals and how many were there of each animal type?

In [5]:
SELECT animal_type, COUNT(DISTINCT animal_id) AS unique_animal
FROM 'animal_shelter.csv'
GROUP BY animal_type
ORDER BY animal_type ASC;

,animal_type,unique_animal
0,Bird,858
1,Cat,64575
2,Dog,78364
3,Livestock,34
4,Other,8722


You may have noticed in our first investigation of the data that some animal names were listed as **null**. This means they didn't have a name or their. name is missing from the records for another reason.

Your turn! How many animals don't have a name?

In [7]:
SELECT COUNT(*) AS animal_no_name
FROM 'animal_shelter.csv'
WHERE name IS NULL;

,animal_no_name
0,48655


Were the animals without a name all a specific animal type?

In [10]:
SELECT animal_type, COUNT(*) AS animal_no_name
FROM 'animal_shelter.csv'
WHERE name IS NULL
GROUP BY animal_type
ORDER BY animal_no_name DESC;

,animal_type,animal_no_name
0,Cat,26270
1,Dog,14510
2,Other,7210
3,Bird,636
4,Livestock,29


Livestock and other seem like broad categories. Do we have any more details on those without a name?

In [13]:
SELECT animal_type, COUNT(breed)
FROM 'animal_shelter.csv'
WHERE animal_type IN ('Livestock', 'Other') AND name IS NULL
GROUP BY animal_type;

,animal_type,count(breed)
0,Other,7210
1,Livestock,29


### Step 3: It's raining cats and dogs

It's probably safe to say that cats and dogs are the most popular pets. But, what is the most common cat breed this shelter saw?

In [25]:
SELECT breed, COUNT(breed) AS breed_count
FROM 'animal_shelter.csv'
WHERE animal_type = 'Cat'
GROUP BY breed
ORDER BY breed_count DESC
LIMIT 10;

,breed,breed_count
0,Domestic Shorthair Mix,34006
1,Domestic Shorthair,22304
2,Domestic Medium Hair Mix,3374
3,Domestic Medium Hair,1999
4,Domestic Longhair Mix,1682
5,Siamese Mix,1461
6,Domestic Longhair,735
7,Siamese,512
8,Snowshoe Mix,236
9,American Shorthair Mix,224


What were the most common dog names in 2023?

In [27]:
SELECT name, COUNT(name) AS count_name
FROM 'animal_shelter.csv'
WHERE animal_type = 'Dog' AND CONTAINS(monthyear, '2023')
GROUP BY name
ORDER BY count_name DESC
LIMIT 10;

,name,count_name
0,Luna,38
1,Max,24
2,Bella,21
3,Milo,17
4,Daisy,17
5,Coco,14
6,Blue,14
7,Zeus,14
8,Buddy,13
9,Apollo,13


How many animals were adopted in 2023?

In [32]:
SELECT outcome_type, COUNT(animal_id) AS count_animal_id
FROM 'animal_shelter.csv'
WHERE CONTAINS (monthyear, '2023') AND outcome_type = 'Adoption'
GROUP BY outcome_type;

,outcome_type,count_animal_id
0,Adoption,6634


Your turn! What was the most adopted animal type and breed in 2023?

In [34]:
SELECT breed, COUNT(animal_id) AS count_animal_id
FROM 'animal_shelter.csv'
WHERE CONTAINS (monthyear, '2023') AND outcome_type = 'Adoption'
GROUP BY breed
ORDER BY count_animal_id DESC
LIMIT 10;

,breed,count_animal_id
0,Domestic Shorthair,2709
1,Domestic Shorthair Mix,280
2,Pit Bull,271
3,Labrador Retriever Mix,262
4,Domestic Medium Hair,215
5,Pit Bull Mix,214
6,German Shepherd Mix,172
7,German Shepherd,129
8,Labrador Retriever,116
9,Domestic Longhair,83
